In [ ]:
import os
import numpy as np
from keras.layers import Input, Dense, Attention
from keras.layers import Input, Dense, LSTM, Reshape
from keras.models import Model
from IPython.display import SVG, clear_output

import keras.callbacks
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# !ls
trainPath = 'train_FD002.txt'
testPath = 'test_FD002.txt'
rulPath = 'RUL_FD002.txt'

In [ ]:
def sliding_window(data, N_tw = 30, stride = 1):
    N_en = np.unique(data[:,0]).shape[0]                            
    m = 0
    for i in range(N_en):
        n_H   = data[data[:,0] == i+1,0].shape[0]
        N_sw  = int((n_H- N_tw) / stride + 1)                       
        for h in range(N_sw):
            m = m + 1    
    return m, N_en        

In [ ]:
train_set=np.loadtxt(trainPath)  # Training set
train_set_cXX = train_set[:,np.array([1])] 
train_set_cXX

In [ ]:
def load_dataset(N_tw, stride, sel, R_early):
    # Load training data
    train_set=np.loadtxt(trainPath)  # Training set
    train_set_x_org = train_set[:,sel]                              
    train_set_c     = train_set[:,np.array([1])]                   
    
    # Normalize the data
    ub = train_set_x_org.max(0)
    lb = train_set_x_org.min(0)    
    train_set_x = 2 * (train_set_x_org - lb) / (ub - lb) - 1   
   
    N_ft    = sel.shape[0]                                           
    m, N_en = sliding_window(train_set, N_tw, stride)               
    
    train_x = np.empty((m, N_tw, N_ft, 1), float)
    train_y = np.empty((m, 1), float)
    
    k = 0
    for i in range(N_en):
        idx       = train_set[:,0] == i+1                           
        train_i_x = train_set_x[idx,:]                              
        train_i_c = train_set_c[idx]                                
        train_i_y = train_i_c[-1] - train_i_c                       
        train_i_y[train_i_y > R_early] = R_early                    
        N_sw      = int((train_i_x.shape[0] - N_tw) / stride + 1)   
        for h in range(N_sw):
            k = k + 1
            vert_start = h * stride
            vert_end   = h * stride + N_tw
            train_i_x_slice = train_i_x[vert_start:vert_end,:]     
            train_i_y_slice = train_i_y[vert_end-1,:]              
            train_i_x_slice.shape = (N_tw, N_ft, 1)                 
            train_i_y_slice.shape = (1, 1)                       
            train_x[k-1,:,:] = train_i_x_slice
            train_y[k-1,:] = train_i_y_slice
     
    # Load test data
    test_set=np.loadtxt(testPath)
    test_set_x_org = test_set[:,sel]                                
    test_set_c     = test_set[:,np.array([1])]                      
    test_y=np.loadtxt(rulPath)   # Test set RUL (c)
    test_y.shape   = (test_y.shape[0], 1)
    
    # Normalize the data
    test_set_x = 2 * (test_set_x_org - lb) / (ub - lb) - 1   
    
    m_ts, N_en_ts = sliding_window(test_set, N_tw, stride)           
    
    test_x = np.empty((N_en_ts, N_tw, N_ft, 1), float)
    
    k = 0
    for ii in range(N_en_ts):
        engine         = test_set[:,0] == ii+1                      
        test_i_x       = test_set_x[engine,:]                       
        test_i_x_slice = test_i_x[-N_tw:,:]                         
        test_i_x_slice.shape = (N_tw, N_ft, 1)                      
        test_x[ii,:,:] = test_i_x_slice
    
    return train_x, train_y, test_x, test_y

In [ ]:
N_tw     = 20                                                              
R_early  = 125                                                             
stride   = 1
sel      = np.array([6, 7, 8, 11, 12, 13, 15, 16, 17, 18, 19, 21, 24, 25]) 

In [ ]:
X_train, Y_train, X_test, Y_test = load_dataset(N_tw, stride, sel, R_early)
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

### 4.3. Model


In [ ]:
# kernels = [10, 10, 10, 10, 3]

# filters = [10, 10, 10, 10, 1]

activ = 'relu'

In [ ]:
def ReshapeLayer(x):
    
    shape = x.shape
    reshape = Reshape((shape[1],shape[2]*shape[3]))(x)
    
    return reshape

In [ ]:
# LSTM + A
def LSTM_Model(input_shape, activ, layer, dropout, encoding_dim):
    # Autoencoder
    # input_autoencoder = Input(shape=input_shape)
    # encoded = Dense(encoding_dim, activation='relu')(input_autoencoder)
    # decoded = Dense(input_shape[0], activation='linear')(encoded)
    # autoencoder = Model(inputs=input_autoencoder, outputs=decoded)

    # LSTM com Autoencoder
    # X_input = Input(input_shape)
    
    # encoded_input = autoencoder(X_input)
    
    X_input = Input(input_shape)
    
    X = Reshape((input_shape[0], -1))(X_input)
    X = LSTM(layer, return_sequences=True, activation=activ, name='lstm0', dropout=dropout)(X)
    X = Attention()([X, X])
    X = Dense(1, activation='linear', name='RUL')(X)

    model = Model(inputs=X_input, outputs=X, name='LSTM_Attention')

    return model

In [ ]:
LSTMAttention = LSTM_Model(X_train.shape[1:], activ, 42, 0, 365)

In [ ]:
LSTMAttention.summary()
keras.utils.plot_model(LSTMAttention, show_shapes=True)

In [ ]:
LSTMAttention.compile(optimizer = keras.optimizers.Adagrad(learning_rate=1e-3), loss = "mean_squared_error")  

In [ ]:
print('Learning Rate: ' + str(K.get_value(LSTMAttention.optimizer.lr)))

In [ ]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []        
        self.fig = plt.figure()        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, np.sqrt(self.losses), label="loss")
        plt.plot(self.x, np.sqrt(self.val_losses), label="val_loss")
        plt.ylabel('loss - RMSE')
        plt.xlabel('epoch')
        plt.legend(['train','test'], loc='upper left')
        plt.title('model loss')
        plt.show();
        
plot_losses = PlotLosses()

In [ ]:
rlr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3)
history = LSTMAttention.fit(X_train, Y_train, epochs = 100, batch_size = 512, shuffle = True, validation_data = (X_test, Y_test), callbacks=[rlr, plot_losses])

In [ ]:
import pickle as pk
history_path = './CNN_FD002_history.pk'
pk.dump(history.history, open(history_path, 'wb'))
model_path = os.path.join('CNN_FD002_model.h5')
LSTMAttention.save(model_path)

In [ ]:
K.set_value(LSTMAttention.optimizer.lr,0.0001)

In [ ]:
LSTMAttention.fit(X_train, Y_train, epochs = 100, batch_size = 512, validation_data = (X_test, Y_test), callbacks=[plot_losses])

In [ ]:
print('Learning Rate: ' + str(K.get_value(LSTMAttention.optimizer.lr)))

In [ ]:
LSTMAttention.save('FD002-CNN.h5') 

In [ ]:
def score_cal(y_hat, Y_test):
    d   = y_hat - Y_test
    tmp = np.zeros(d.shape[0])
    for i in range(d.shape[0]):
        if d[i,0] >= 0:
               tmp[i] = np.exp( d[i,0]/10) - 1
        else:
               tmp[i] = np.exp(-d[i,0]/13) - 1
    return tmp 

In [ ]:
preds = LSTMAttention.evaluate(x = X_train, y = Y_train)
print()
print ("Test  MSE = " + str(preds))
print ("Test RMSE = " + str(np.sqrt(preds)))

In [ ]:
y_hat_tr   = LSTMAttention.predict(x = X_train)

In [ ]:
preds = LSTMAttention.evaluate(x = X_test, y = Y_test)
print()
print ("MSE = " + str(preds))
print ("RMSE = " + str(np.sqrt(preds)))